In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
sys.path.append("../../../src/")
sys.path.append("../..")

import torch
import torchvision
import numpy as np
from torchvision import transforms

from models.GuidedReLu import GuidedBackpropReLU
from loss import FilterLoss
import model

In [6]:
selected_layer = 1

In [7]:
resume = "037-0"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
backbone = "vgg16"
# Load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = model.Network(backbone=backbone, num_classes=1000, selected_layer=selected_layer)
net.to(device)
# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt, False) 

Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0


_IncompatibleKeys(missing_keys=[], unexpected_keys=['model.fc.0.weight', 'model.fc.0.bias', 'model.fc.3.weight', 'model.fc.3.bias', 'model.fc.6.weight', 'model.fc.6.bias', 'model.features.2.weight', 'model.features.2.bias', 'model.features.5.weight', 'model.features.5.bias', 'model.features.7.weight', 'model.features.7.bias', 'model.features.10.weight', 'model.features.10.bias', 'model.features.12.weight', 'model.features.12.bias', 'model.features.14.weight', 'model.features.14.bias', 'model.features.17.weight', 'model.features.17.bias', 'model.features.19.weight', 'model.features.19.bias', 'model.features.21.weight', 'model.features.21.bias', 'model.features.24.weight', 'model.features.24.bias', 'model.features.26.weight', 'model.features.26.bias', 'model.features.28.weight', 'model.features.28.bias'])

In [9]:
net.eval()

Network(
  (model): VGG16(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
  )
)

In [10]:
net.set_guildedReLU(True)

Using GuidedReLu.


In [11]:
net

Network(
  (model): VGG16(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): <built-in method apply of FunctionMeta object at 0x55f6b5979258>
    )
  )
)

In [12]:
inputs = torch.randn(2,3,224,224).cuda()
inputs2 = torch.randn_like(inputs).cuda()
out = net(inputs)

In [13]:
criterion = FilterLoss(net, selected_layer, selected_filter=112, mode="keep", _print=print)

In [14]:
selected_filter_loss, rest_fileter_loss, regularization_loss,  \
    smoothing_loss = criterion(inputs, inputs2)
# if alpha != 0 and (1-alpha) != 0:
# use beat to omit gradient from rest_filter_loss
loss = selected_filter_loss + rest_fileter_loss + \
    regularization_loss 

IndexError: index 112 is out of bounds for dimension 1 with size 64

In [ ]:
loss.backward()

In [ ]:
selec